<a href="https://colab.research.google.com/github/setthawut8/ai/blob/main/%5BEmbedding_Layer%2BCNN%5D_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Inspiration
https://machinelearningmastery.com/develop-word-embedding-model-predicting-movie-review-sentiment/

=================================================================
https://pythainlp.github.io/tutorials/notebooks/word2vec_examples.html

+

https://ml2021.medium.com/multi-class-text-classification-using-cnn-and-word2vec-b17daff45260

(https://github.com/ML2021/TF-RNN-CNN-text-processing/blob/main/Multi%20Class%20Text%20Classification%20word2vec%20using%20CNN.ipynb)

#1) Download and Installation

In [1]:
!wget "https://raw.githubusercontent.com/jbrownlee/Datasets/master/review_polarity.tar.gz"

--2022-07-01 07:44:35--  https://raw.githubusercontent.com/jbrownlee/Datasets/master/review_polarity.tar.gz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3127238 (3.0M) [application/octet-stream]
Saving to: ‘review_polarity.tar.gz’

review_polarity.tar 100%[===================>]   2.98M  --.-KB/s    in 0.04s   

2022-07-01 07:44:35 (69.4 MB/s) - ‘review_polarity.tar.gz’ saved [3127238/3127238]



In [2]:
!tar -xf '/content/review_polarity.tar.gz'

In [3]:
import string
import os, os.path
import pandas as pd
import seaborn as sns

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [4]:
print("neg: ",len(os.listdir('/content/txt_sentoken/neg')), " pos: ", len(os.listdir('/content/txt_sentoken/pos')))

neg:  1000  pos:  1000


#2) Loading and Cleaning Reviews

In [5]:
#Open and return text from a directory
def load_doc(filename):
  file = open(filename, 'r')
  text = file.read()
  file.close()
  return text

#Turn doc to cleaned tokens
def clean_doc(doc):
  tokens = doc.split()
  table = str.maketrans("","",string.punctuation)
  tokens = [w.translate(table) for w in tokens]
  tokens = [word for word in tokens if word.isalpha()]
  stop_words = set(stopwords.words('english'))
  tokens = [w for w in tokens if not w in stop_words]
  tokens = [word for word in tokens if len(word)>1]
  return tokens

#load the document
filename = 'txt_sentoken/pos/cv000_29590.txt'
text = load_doc(filename)
tokens = clean_doc(text)
print(tokens)

['films', 'adapted', 'comic', 'books', 'plenty', 'success', 'whether', 'theyre', 'superheroes', 'batman', 'superman', 'spawn', 'geared', 'toward', 'kids', 'casper', 'arthouse', 'crowd', 'ghost', 'world', 'theres', 'never', 'really', 'comic', 'book', 'like', 'hell', 'starters', 'created', 'alan', 'moore', 'eddie', 'campbell', 'brought', 'medium', 'whole', 'new', 'level', 'mid', 'series', 'called', 'watchmen', 'say', 'moore', 'campbell', 'thoroughly', 'researched', 'subject', 'jack', 'ripper', 'would', 'like', 'saying', 'michael', 'jackson', 'starting', 'look', 'little', 'odd', 'book', 'graphic', 'novel', 'pages', 'long', 'includes', 'nearly', 'consist', 'nothing', 'footnotes', 'words', 'dont', 'dismiss', 'film', 'source', 'get', 'past', 'whole', 'comic', 'book', 'thing', 'might', 'find', 'another', 'stumbling', 'block', 'hells', 'directors', 'albert', 'allen', 'hughes', 'getting', 'hughes', 'brothers', 'direct', 'seems', 'almost', 'ludicrous', 'casting', 'carrot', 'top', 'well', 'anythi

#3) Defind a Vocabulary

In [62]:
from numpy import array

from string import punctuation
from os import listdir
from collections import Counter
from nltk.corpus import stopwords

from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences

from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Embedding
from keras.models import Sequential
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D

In [7]:
#Create a desired vocabulary

#load doc into memory
def load_doc(filename):
  file = open(filename, 'r')
  text = file.read()
  file.close()
  return text

def clean_doc(doc):
  tokens = doc.split()
  table = str.maketrans('', '', punctuation)
  tokens = [w.translate(table) for w in tokens]
  tokens = [word for word in tokens if word.isalpha()]
  stop_words = set(stopwords.words('english'))
  tokens = [word for word in tokens if not word in stop_words]
  tokens = [word for word in tokens if len(tokens) > 1]
  return tokens

def add_doc_to_vocab(filename, vocab):
  doc = load_doc(filename)
  tokens = clean_doc(doc)
  vocab.update(tokens)

def process_docs(directory, vocab, is_train):
  for filename in listdir(directory):
    if is_train and filename.startswith('cv9'):
      continue
    if not is_train and not filename.startswith('cv9'):
      continue
    path = directory + '/' + filename
    add_doc_to_vocab(path, vocab)

vocab = Counter()
process_docs('txt_sentoken/neg', vocab, True)
process_docs('txt_sentoken/pos', vocab, True)

print(len(vocab))
print(vocab.most_common(50))

44294
[('film', 7983), ('one', 4946), ('movie', 4826), ('like', 3201), ('even', 2262), ('good', 2080), ('time', 2041), ('story', 1907), ('films', 1873), ('would', 1844), ('much', 1824), ('also', 1757), ('characters', 1735), ('get', 1724), ('character', 1703), ('two', 1643), ('first', 1588), ('see', 1557), ('way', 1515), ('well', 1511), ('make', 1418), ('really', 1407), ('little', 1351), ('life', 1334), ('plot', 1288), ('people', 1269), ('bad', 1248), ('could', 1248), ('scene', 1241), ('movies', 1238), ('never', 1201), ('best', 1179), ('new', 1140), ('scenes', 1135), ('man', 1131), ('many', 1130), ('doesnt', 1118), ('know', 1092), ('dont', 1086), ('hes', 1024), ('great', 1014), ('another', 992), ('action', 985), ('love', 977), ('us', 967), ('go', 952), ('director', 948), ('end', 946), ('something', 945), ('still', 936)]


In [8]:
min_occurrence = 2
tokens = [k for k, c in vocab.items() if c >= min_occurrence]
print(len(tokens))

25785


In [9]:
def save_list(lines, filename):
  data = '\n'.join(lines)
  file = open(filename, 'w')
  file.write(data)
  file.close()

save_list(tokens, 'vocab.txt')

#4) Train Embedding Layer

In [10]:
def load_doc(filename):
  file = open(filename, 'r')
  text = file.read()
  file.close()
  return text

vocab_filename = 'vocab.txt'
vocab = load_doc(vocab_filename)
vocab = vocab.split()
vocab = set(vocab)
print(vocab)

{'central', 'bancroft', 'stalker', 'addition', 'puking', 'illiterate', 'flashdance', 'busey', 'courtney', 'prevents', 'pedro', 'noisy', 'enthralling', 'brandos', 'insulting', 'hayeks', 'initially', 'divorcing', 'sound', 'fancy', 'mouthpieces', 'everyman', 'david', 'helgelands', 'backup', 'fumes', 'marianne', 'schwarzeneggers', 'ore', 'premed', 'scummy', 'spiritually', 'werewolf', 'kotto', 'accident', 'p', 'gold', 'fannys', 'lovesick', 'mobutu', 'paced', 'democratic', 'rainforest', 'fernanda', 'kalvert', 'cited', 'kirkland', 'eliminate', 'plenty', 'hilt', 'bunz', 'ceremonies', 'wessell', 'kitano', 'instrument', 'skillfully', 'narrating', 'riddles', 'underdog', 'marseilles', 'hun', 'announces', 'clues', 'prolonged', 'abba', 'robicheaux', 'niche', 'humans', 'invent', 'atmospheric', 'freddie', 'brim', 'inquired', 'applicable', 'sheikhi', 'biographies', 'asbestos', 'luckily', 'terrorized', 'impervious', 'glove', 'leone', 'mcdonalds', 'seth', 'disappointment', 'translator', 'anij', 'resembli

In [49]:
def clean_doc(doc, vocab):
  tokens = doc.split()
  table = str.maketrans('', '', punctuation)
  tokens = [w.translate(table) for w in tokens]
  tokens = [w for w in tokens if w in vocab]
  tokens = ' '.join(tokens)
  return tokens

def process_docs(directory, vocab, is_train):
  documents = list()
  for filename in listdir(directory):
    if is_train and filename.startswith('cv9'):
        continue
    if not is_train and not filename.startswith('cv9'):
        continue
    path = directory + '/' + filename
    doc = load_doc(path)
    tokens = clean_doc(doc, vocab)
    documents.append(tokens)
  return documents

positive_docs = process_docs('txt_sentoken/pos', vocab, True)
negative_docs = process_docs('txt_sentoken/neg', vocab, True)

train_docs = negative_docs + positive_docs

In [50]:
train_docs[0]

'amazing comedian funniest standup around completely fall flat movies take adam sandler basis first two cds theyre gonna laugh hell happened hes best thing hit comedy since robin williams put front camera isnt kind enjoyed happy gilmore thought warranted marginal recommendation latest film waterboy funny root canal seriously im willing wager laughs saving private ryan fact found movie depressing comedic first half hour generated one smile scenes emotional abuse towards adams character bobby cruel football players also disturbed mother kathy bates world coming granted film gets rolling mild chuckles throughout im even willing admit actually laughing twice nearly enough sustain waterboy nearly hour half running time youre familiar plot youve seen least million times everyone picks hero shows hidden talent leads underdog team victory champions process winning respect teammates also getting woman loves believe im giving anything away saying sandlers team wins would actually feel degree sus

In [51]:
#Encode each document as a sequence of integers
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_docs)
encoded_docs = tokenizer.texts_to_sequences(train_docs)

In [52]:
#as each sentences contain different length of words, we need padding to make the sequences the same length before
#putting into the embedding layer
max_length = max([len(s.split()) for s in train_docs])
Xtrain = pad_sequences(encoded_docs, maxlen=max_length, padding='post')

In [53]:
#define the labels for training data sets
ytrain = array([0 for _ in range(900)] + [1 for _ in range(900)])

In [56]:
#do as processes above for the testing set.
positive_docs = process_docs('txt_sentoken/pos', vocab, False)
negative_docs = process_docs('txt_sentoken/neg', vocab, False)
test_docs = negative_docs + positive_docs

encoded_docs = tokenizer.texts_to_sequences(test_docs)
Xtest = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
ytest = array([0 for _ in range(100)] + [1 for _ in range(100)])

In [57]:
#defind vocabulary size
vocab_size = len(tokenizer.word_index) + 1

#5) Model

In [65]:
#define model
model = Sequential()
model.add(Embedding(vocab_size, 100, input_length=max_length))
model.add(Conv1D(filters=32, kernel_size=8, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(10, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 1319, 100)         2578600   
                                                                 
 conv1d_1 (Conv1D)           (None, 1312, 32)          25632     
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 656, 32)          0         
 1D)                                                             
                                                                 
 flatten_1 (Flatten)         (None, 20992)             0         
                                                                 
 dense_2 (Dense)             (None, 10)                209930    
                                                                 
 dense_3 (Dense)             (None, 1)                 11        
                                                      

In [66]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(Xtrain, ytrain, epochs=10, verbose=2)

Epoch 1/10
57/57 - 16s - loss: 0.6912 - accuracy: 0.5267 - 16s/epoch - 277ms/step
Epoch 2/10
57/57 - 14s - loss: 0.5920 - accuracy: 0.7100 - 14s/epoch - 253ms/step
Epoch 3/10
57/57 - 14s - loss: 0.1192 - accuracy: 0.9761 - 14s/epoch - 252ms/step
Epoch 4/10
57/57 - 14s - loss: 0.0078 - accuracy: 1.0000 - 14s/epoch - 243ms/step
Epoch 5/10
57/57 - 14s - loss: 0.0023 - accuracy: 1.0000 - 14s/epoch - 238ms/step
Epoch 6/10
57/57 - 14s - loss: 0.0014 - accuracy: 1.0000 - 14s/epoch - 240ms/step
Epoch 7/10
57/57 - 15s - loss: 9.8218e-04 - accuracy: 1.0000 - 15s/epoch - 263ms/step
Epoch 8/10
57/57 - 15s - loss: 7.3593e-04 - accuracy: 1.0000 - 15s/epoch - 260ms/step
Epoch 9/10
57/57 - 14s - loss: 5.6691e-04 - accuracy: 1.0000 - 14s/epoch - 241ms/step
Epoch 10/10
57/57 - 14s - loss: 4.4245e-04 - accuracy: 1.0000 - 14s/epoch - 238ms/step


#6) Evaluation

In [68]:
loss, acc = model.evaluate(Xtest, ytest, verbose=0)
print('Test Accuracy: %f' % (acc*100))

Test Accuracy: 88.999999
